<a href="https://colab.research.google.com/github/srikarraju/NLP_Project/blob/main/ADL_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#replace all emojis with phrases
import re
def replace_emojis(text):
  happyface = ":\)"
  text = re.sub(r"http.*","url",text)
  text = re.sub(happyface,"happyface",text)
  happyface = ":-\)"
  text = re.sub(happyface,"happyface",text)
  happyface = ":-\]"
  text = re.sub(happyface,"happyface",text)
  happyface = ":\]"
  text = re.sub(happyface,"happyface",text)
  happyface = "=\)"
  text = re.sub(happyface,"happyface",text)
  happyface = ":\'-\)"
  text = re.sub(happyface,"happyface",text)
  happyface = ":\'\)"
  text = re.sub(happyface,"happyface",text)
  sadface = ":\("
  text = re.sub(sadface,"sadface",text)
  sadface = ":-\("
  text = re.sub(sadface,"sadface",text)
  sadface = ":\["
  text = re.sub(sadface,"sadface",text)
  sadface = ":-\["
  text = re.sub(sadface,"sadface",text)
  sadface = ":\'-\("
  text = re.sub(sadface,"sadface",text)
  sadface = ":\'\("
  text = re.sub(sadface,"sadface",text)
  sadface = "\(:"
  text = re.sub(sadface,"sadface",text)
  # text = re.sub(":|","neutralface",text)
  text = re.sub(":P","lolface",text)
  text = re.sub(":p","lolface",text)
  text = re.sub(":o","surprise",text)
  text = re.sub(":O","surprise",text)
  #print(text[0:1000])
  return text

In [2]:
def split_into_words(text):
  sentences = text.splitlines()
  words_sentences = []
  for sent in sentences:
    #words = sent.split("\t")
    words = sent.split()
    words_sentences.append(words)
  #print(words_sentences[0:10])
  return words_sentences

In [3]:
def seperate_classes_ids_sentences(word_sentences):
  sentence_ids = []
  sentence_classes = []
  sentences = []
  for sent in word_sentences:
    if len(sent)==0:
      continue
    if sent[0].isdigit():
      sentence_ids.append(int(sent[0]))
    else:
      continue
    sentence_classes.append(sent[1])
    words = sent[2:]
    #print(words)
    sentences.append(words)
  print(sentence_ids)
  print(sentence_classes)
  print(sentences[0:5])
  return sentences,sentence_classes,sentence_ids

In [4]:
import re
def remove_punctuations(sentences):
  def remove_punctuations_word(input_string):
    input_string = re.sub(r'\\u002c','',input_string)
    input_string = re.sub(r'@.*','',input_string)
    input_string = re.sub(r'#.*',' ',input_string)
    input_string = re.sub(r'[0-9]+', '', input_string)
    res = re.sub(r'[^\w\s]', '', input_string)
    return res
  for sent in sentences:
    i = 0
    while i<len(sent):
      word = remove_punctuations_word(sent[i])
      sent[i] = word
      if len(sent[i])==0:
        sent.pop(i)
        i -= 1 
      i += 1
  print(sentences[0:10])
  return sentences

In [5]:
stopwords = ["ourselves", "hers", "between", "yourself","again", "there", "about", "once", "during", "out", "having",
             "with", "they", "own", "an", "be", "some", "for", "do", "its", "yours", "such", "into", "of", "itself", "other",
             "off", "is", "am", "or", "who", "as", "from", "him", "each", "the", "themselves", "until", "below", "are", "we",
             "these", "your", "his", "through", "don", "nor", "me", "were", "her", "more", "himself", "this", "down", "should", "our",
             "their", "while", "above", "both", "up", "to", "ours", "had", "she", "all", "when", "at", "any", "before", "them",
             "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", "then", "that", "what", "over", "why",
             "so", "can", "did", "now", "under", "he", "you", "herself", "has",  "where", "only", "myself", 
             "which", "those", "i", "after", "whom", "to", "being", "if", "theirs", "my", "against", "a", "by", "doing", "it",
             "how", "further", "was", "here", "than","one","time"]
def remove_stopwords(words_sentences):
  filtered_words = []
  for sentence in words_sentences:
    filtered_sentence = []
    for word in sentence:
      if word not in stopwords:
        filtered_sentence.append(word)
    filtered_words.append(filtered_sentence)
  return filtered_words

In [6]:
def remove_recurring_characters(sentences):
  def remove_recurring_characters_in_word(word):
    i = 1
    rec_count = 0
    while i < len(word):
      if word[i] == word[i-1]:
        rec_count += 1
      else:
        rec_count =0
      if rec_count>=2:
        word = word[:i]+word[i+1:]
        i -= 1
      i += 1
    return word
  for sent in sentences:
    for i in range(len(sent)):
      sent[i] = remove_recurring_characters_in_word(sent[i]).lower()
  print(sentences[0:10])
  return sentences

In [7]:
def get_train_sentence_vectors(sentences,model):
  words_to_integer_dict = dict()
  integer_to_word_dict = dict()
  print("No. of sentences = ",len(sentences))
  sentence_vectors = []
  max_sentence_length = 0
  word_count = 1
  for sent in sentences:
    vec = []
    sent_length = 0
    for word in sent:
      sent_length += 1
      if words_to_integer_dict.get(word) == None:
        if word in model:
          words_to_integer_dict[word] = word_count
          vec.append(word_count)
          word_count += 1
        else:
          vec.append(0)
      else:
        vec.append(words_to_integer_dict[word])
    max_sentence_length = max(sent_length,max_sentence_length)
    sentence_vectors.append(vec)
  return sentence_vectors,word_count,max_sentence_length,words_to_integer_dict

In [8]:
def get_embeds_matrix(embeds_list):
  embeds_matrix = []
  for i in range(len(embeds_list)):
    embeds_list[i] = str(embeds_list[i][1:len(embeds_list[i])-1])
    array = embeds_list[i].split()
    for j in range(0,len(array)):
      array[j] = float(array[j])
    embeds_matrix.append(array)
  
  return np.asarray(embeds_matrix)

In [9]:
#initializing the embedding matrix for embedding layer in the model
import numpy as np
import pandas as pd
def get_initial_embeddings(model_ft,word_count,words_to_integer_dict):
  embeds_file = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/Project/distant_training_embeddings.csv")
  count_in_model = 0
  embedding_matrix = [[0]*50]*(word_count+2)  #one for not in model one for not in train data
  words_list = embeds_file['words'].tolist()
  embeds_list = embeds_file['embeds'].tolist()
  embeds_matrix = get_embeds_matrix(embeds_list)
  for word in words_to_integer_dict:
    if word in words_list:
      embedding_matrix[words_to_integer_dict[word]] = np.asarray(embeds_matrix[words_list.index(word)])
      count_in_model += 1
    elif word in model_ft:
      count_in_model += 1
      embedding_matrix[words_to_integer_dict[word]] = model[word]
  embedding_matrix = np.asarray(embedding_matrix,dtype=float)
  print("No. of embeddings in model = ",count_in_model)
  return embedding_matrix

In [10]:
def get_test_sentence_vectors(sentences,word_count):
  sentence_vectors = []
  for sent in sentences:
    vec = []
    for word in sent:
      if words_to_integer_dict.get(word) != None:
        vec.append(words_to_integer_dict[word])
      else:
        vec.append(word_count+1)
    sentence_vectors.append(vec)
  return sentence_vectors

In [11]:
def pad_input_vectors(sentence_vectors,max_sentence_length):
  for i in range(len(sentence_vectors)):
    sentence_vectors[i] = sentence_vectors[i][0:max_sentence_length]
    for j in range(max_sentence_length-len(sentence_vectors[i])):
      sentence_vectors[i].append(0)
  #print(sentence_vectors[0:2])
  data_x = np.asarray(sentence_vectors)
  return data_x

In [12]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def get_input_class_labels(sentence_classes):
  le.fit(sentence_classes)
  data_y = le.transform(sentence_classes)
  data_y = np.asarray(data_y)
  return data_y
def get_test_class_labels(sentence_classes):
  test_y = le.transform(sentence_classes)
  return np.asarray(test_y)

In [13]:
from sklearn.model_selection import train_test_split
def split_train_data(data_x,data_y):
  train_x,dev_x,train_y,dev_y = train_test_split(data_x,data_y,test_size=0.25,random_state=20,stratify=data_y)
  return train_x,dev_x,train_y,dev_y

In [14]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Softmax, Bidirectional, Flatten, Average
from keras.models import Sequential,Model
import tensorflow as tf

def get_weights(train_y):
  count = np.zeros(3,dtype = float)
  for i in range(len(train_y)):
    count[train_y[i]] += 1
  for i in range(3):
    count[i] = 1/count[i]
  weights = []
  for i in range(len(train_y)):
    weights.append(count[train_y[i]])
  weights = np.asarray(weights)
  return weights

def get_trained_LSTM_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length):
  initializer = tf.keras.initializers.HeNormal()
  weights = get_weights(train_y)
  model1 = Sequential()
  e = Embedding(word_count+2, 50, weights = [embedding_matrix], input_length=max_sentence_length)
  e.trainable = False
  model1.add(e)
  model1.add(Dropout(0.5))
  model1.add(Bidirectional(LSTM(100,activation="tanh",recurrent_dropout=0.5,recurrent_activation="tanh")))
  model1.add(Flatten())#using all lstm hidden state ouputs
  model1.add(Dropout(0.5))
  model1.add(Dense(50,activation="relu",kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4)))
  model1.add(Dropout(0.5))
  model1.add(Dense(3,activation="softmax"))

  model1.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  model1.fit(train_x,train_y,epochs=1)
  e.trainable = True
  model1.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  model1.fit(train_x,train_y,epochs=4)
  return model1
  # predictions = model1.predict(test_x)

In [15]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Softmax, Bidirectional, Flatten, Average, GRU
from keras.models import Sequential,Model
import tensorflow as tf

def get_weights(train_y):
  count = np.zeros(3,dtype = float)
  for i in range(len(train_y)):
    count[train_y[i]] += 1
  for i in range(3):
    count[i] = 1/count[i]
  weights = []
  for i in range(len(train_y)):
    weights.append(count[train_y[i]])
  weights = np.asarray(weights)
  return weights

def get_trained_GRU_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length):
  initializer = tf.keras.initializers.HeNormal()
  weights = get_weights(train_y)
  model1 = Sequential()
  e = Embedding(word_count+2, 50, weights = [embedding_matrix], input_length=max_sentence_length)
  e.trainable = False
  model1.add(e)
  model1.add(Dropout(0.5))
  model1.add(Bidirectional(GRU(100,activation="tanh",recurrent_dropout=0.5,recurrent_activation="tanh")))
  model1.add(Flatten())#using all lstm hidden state ouputs
  model1.add(Dropout(0.5))
  model1.add(Dense(50,activation="relu",kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4)))
  model1.add(Dropout(0.5))
  model1.add(Dense(3,activation="softmax"))

  model1.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  model1.fit(train_x,train_y,epochs=1)
  e.trainable = True
  model1.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  model1.fit(train_x,train_y,epochs=4)
  return model1
  # predictions = model1.predict(test_x)

In [ ]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Softmax, Bidirectional, Flatten, Average, SimpleRNN
from keras.models import Sequential,Model
import tensorflow as tf

def get_RNN_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length):
  initializer = tf.keras.initializers.HeNormal()
  model1 = Sequential()
  e = Embedding(word_count+2, 50, weights = [embedding_matrix], input_length=max_sentence_length)
  e.trainable = False
  model1.add(e)
  model1.add(Dropout(0.5))
  model1.add(Bidirectional(SimpleRNN(100,activation="tanh",recurrent_dropout=0.5)))
  model1.add(Flatten())#using all lstm hidden state ouputs
  model1.add(Dropout(0.5))
  model1.add(Dense(50,activation="relu",kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4)))
  model1.add(Dropout(0.5))
  model1.add(Dense(3,activation="softmax"))

  model1.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  model1.fit(train_x,train_y,epochs=2)
  #e.trainable = True
  model1.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  model1.fit(train_x,train_y,epochs=4)
  return model1
  # predictions = model1.predict(test_x)

In [ ]:
from keras.layers import Lambda
import keras.backend as K

def get_weights(train_y):
  count = np.zeros(3,dtype = float)
  for i in range(len(train_y)):
    count[train_y[i]] += 1
  for i in range(5):
    count[i] = len(train_y)/count[i]
  weights = []
  for i in range(len(train_y)):
    weights.append(count[train_y[i]])
  weights = np.asarray(weights)
  return weights

def get_modified_LSTM_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length,dev_x,dev_y):
  #weights = get_weights(train_y)
  #print(weights)
  initializer = tf.keras.initializers.HeNormal()
  model2 = Sequential()
  e = Embedding(word_count+2, 50, weights = [embedding_matrix], input_length=max_sentence_length)
  model2.add(e)
  model2.add(Dropout(0.5))
  model2.add(Bidirectional(LSTM(100,activation="tanh",recurrent_dropout=0.5,recurrent_activation="tanh",return_sequences=True)))
  model2.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(200,)))#using all lstm hidden state ouputs
  model2.add(Dropout(0.5))
  model2.add(Dense(50,activation="relu"))
  model2.add(Dropout(0.5))
  model2.add(Dense(3,activation="softmax"))

  model2.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  model2.fit(train_x,train_y,epochs=4,validation_data=(dev_x,dev_y))
  # model2.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  # model2.fit(train_x,train_y,epochs=4)
  return model2

In [ ]:
class Attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(Attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(Attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [ ]:
from keras.layers import Lambda
import keras.backend as K

def get_weights(train_y):
  count = np.zeros(3,dtype = float)
  for i in range(len(train_y)):
    count[train_y[i]] += 1
  for i in range(3):
    count[i] = 1/count[i]
  weights = []
  for i in range(len(train_y)):
    weights.append(count[train_y[i]])
  weights = np.asarray(weights)
  return weights

def get_LSTM_Attn_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length,dev_x,dev_y):
  weights = get_weights(train_y)
  #print(weights)
  initializer = tf.keras.initializers.HeNormal()
  model2 = Sequential()
  e = Embedding(word_count+2, 50, weights = [embedding_matrix], input_length=max_sentence_length)
  model2.add(e)
  model2.add(Dropout(0.5))
  model2.add(Bidirectional(LSTM(100,activation="tanh",recurrent_dropout=0.5,recurrent_activation="tanh",return_sequences=True)))
  model2.add(Attention(return_sequences=False))#using all lstm hidden state ouputs
  model2.add(Dropout(0.5))
  model2.add(Dense(50,activation="relu"))
  model2.add(Dropout(0.5))
  model2.add(Dense(3,activation="softmax"))

  model2.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'],loss_weights=weights)
  model2.fit(train_x,train_y,epochs=4,validation_data=(dev_x,dev_y))
  # model2.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  # model2.fit(train_x,train_y,epochs=4)
  return model2

In [ ]:
# CNN model
from keras.layers import Conv2D, MaxPooling2D, Concatenate, Reshape
import tensorflow as tf

def get_weights(train_y):
  print("getting weights")
  count = np.zeros(5,dtype = float)
  for i in range(len(train_y)):
    count[train_y[i]] += 1
  for i in range(5):
    count[i] = len(train_y)/count[i]
  weights = []
  for i in range(len(train_y)):
    weights.append(count[train_y[i]])
  weights = np.asarray(weights)
  return weights

def get_trained_CNN_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length):
  weights = get_weights(train_y)
  initializer = tf.keras.initializers.HeNormal()
  input = Input(shape=(max_sentence_length,))
  e = Embedding(word_count+2, 50, weights = [embedding_matrix], input_length=max_sentence_length)(input)
  e.trainable = False
  reshaped_embedding = Reshape((1,max_sentence_length,50,1))(e)
  filter_2 = Conv2D(50,(2,50),activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(reshaped_embedding)
  filter_3 = Conv2D(50,(3,50),activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(reshaped_embedding) 
  filter_4 = Conv2D(50,(4,50),activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(reshaped_embedding)
  reshaped_filter_2 = Reshape((max_sentence_length-1,50,1))(filter_2)
  reshaped_filter_3 = Reshape((max_sentence_length-2,50,1))(filter_3)
  reshaped_filter_4 = Reshape((max_sentence_length-3,50,1))(filter_4)
  max_pooling_2 = MaxPooling2D(pool_size=(max_sentence_length-1,1))(reshaped_filter_2)
  max_pooling_3 = MaxPooling2D(pool_size=(max_sentence_length-2,1))(reshaped_filter_3)
  max_pooling_4 = MaxPooling2D(pool_size=(max_sentence_length-3,1))(reshaped_filter_4)
  concatenated_output = Concatenate()([max_pooling_2,max_pooling_3,max_pooling_4])
  concatenated_output_reshaped = Reshape((1,150))(concatenated_output)
  dense1_output = Dense(50,activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(concatenated_output_reshaped)
  dense2_output = Dense(3,activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(dense1_output)
  softmax_probs = Softmax()(dense2_output)

  model3 = Model(input,softmax_probs)
  print(model3.summary())
  print("compiling model")
  model3.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  print("fitting model")
  model3.fit(train_x,train_y,epochs=2)
  e.trainable = True
  model3.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  model3.fit(train_x,train_y,epochs=4)
  return model3

In [19]:
#Converting the softmax outputs to their respective classes 
def get_predictions_single_model(model,test_x):
  predictions = model.predict(test_x)
  predictions = np.reshape(predictions,(len(test_x),3))
  print(predictions)
  predict_classes = []
  for i in range(len(predictions)):
    for j in range(3):
      if predictions[i][j] == max(predictions[i]):
        predict_classes.append(j)
        break
  predictions = predict_classes
  print(predictions)
  return predictions

In [22]:
# MACRO-AVG-RECALL-METRIC
def calculate_avg_recall(predictions,test_y):
  true_pos_predcited,total_true_pos = 0,0
  true_neg_predcited,total_true_neg = 0,0
  true_neu_predcited,total_true_neu = 0,0
  for i in range(len(predictions)):
    if test_y[i] == 2:
      total_true_pos += 1
      if predictions[i] == 2:
        true_pos_predcited += 1
    elif test_y[i] == 0:
      total_true_neg += 1
      if predictions[i] == 0:
        true_neg_predcited += 1
    elif test_y[i] == 1:
      total_true_neu += 1
      if predictions[i] == 1:
        true_neu_predcited += 1
  print(true_pos_predcited/total_true_pos,true_neg_predcited/total_true_neg,true_neu_predcited/total_true_neu)
  print(sum([true_pos_predcited/total_true_pos,true_neg_predcited/total_true_neg,true_neu_predcited/total_true_neu])/3)
  return sum([true_pos_predcited/total_true_pos,true_neg_predcited/total_true_neg,true_neu_predcited/total_true_neu])/3

In [ ]:
def calculate_avg_recall_2(predictions,test_y):
  total_class_predicted = np.zeros(5)
  total_true_class = np.zeros(5)
  for i in range(len(predictions)):
    if test_y[i] == predictions[i]:
      total_class_predicted[int(predictions[i])] += 1
    total_true_class[int(test_y[i])] += 1
  recall = []
  for i in range(5):
    recall.append(total_class_predicted[i]/total_true_class[i])
  print(recall)
  return sum(recall)/5

In [16]:
from gensim.models import Word2Vec,KeyedVectors
import gensim.downloader
#model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/NLP/GoogleNews_Vectors_300.bin', binary=True)
model = gensim.downloader.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded


In [17]:
#Read the input file
train_file = open("/content/drive/MyDrive/Datasets/train_2016",'r',encoding='utf-8')
text = train_file.read()
train_file.close()

text = replace_emojis(text)
word_sentences = split_into_words(text)
text = None
word_sentences = remove_stopwords(word_sentences)
print(word_sentences[0:2])
sentences,sentence_classes,sentence_ids = seperate_classes_ids_sentences(word_sentences)
sentences = remove_punctuations(sentences)
word_sentences = None
sentences = remove_recurring_characters(sentences)
print("Creating sentence_vectors")
sentence_vectors,word_count,max_sentence_length,words_to_integer_dict = get_train_sentence_vectors(sentences,model)
sentences = None

#max_sentence_length = 512

print("Word count,Max Sentence Length = ",word_count,max_sentence_length)

data_x = pad_input_vectors(sentence_vectors,max_sentence_length)

#train_file = open("/content/drive/My Drive/Colab Notebooks/NLP/Assignment-3/digital_music_5.y.train.txt",'r',encoding='utf-8')
#output_labels = train_file.readlines()
#train_file.close()
data_y = get_input_class_labels(sentence_classes)
sentence_classes = None

#train_x,dev_x,train_y,dev_y = split_train_data(data_x,data_y)
#data_x,data_y = None,None

print("Creating Embeddings")

embedding_matrix = get_initial_embeddings(model,word_count,words_to_integer_dict)



[['264183816548130816', 'positive', 'Gas', 'house', 'hit', '$3.39!!!!', 'I\\u2019m', 'going', 'Chapel', 'Hill', 'Sat.', 'happyface'], ['263405084770172928', 'negative', 'Theo', 'Walcott', 'still', 'shit\\u002c', 'watch', 'Rafa', 'Johnny', 'deal', 'Saturday.']]
[264183816548130816, 263405084770172928, 262163168678248449, 264249301910310912, 262682041215234048, 264229576773861376, 264105751826538497, 264094586689953794, 212392538055778304, 254941790757601280, 264169034155696130, 263192091700654080, 263398998675693568, 263650552167157762, 260200142420992000, 264087629237202944, 250692636330049538, 263304719471087617, 261954070938537985, 260940907082293248, 263956867787673600, 263975113404342273, 257343699460173824, 264125591337463808, 262350309781823488, 264259830590603264, 257239661976625152, 263868270006906880, 264223934403211264, 264041764460036096, 264102295392882689, 264173636141850626, 264071572925603841, 262115068320284672, 264239636799430656, 264250109833924608, 257951107530252289

In [ ]:
LSTM_model1 = get_trained_LSTM_model(data_x,data_y,embedding_matrix,word_count,max_sentence_length)
predictions_LSTM_model1 = get_predictions_single_model(LSTM_model1,dev_x)
LSTM1_avg_recall = calculate_avg_recall(predictions_LSTM_model1,dev_y)

print(LSTM1_avg_recall)

getting weights


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


1313/1313 [==============================] - 235s 175ms/step - loss: 1.0281 - accuracy: 0.4585
Epoch 1/4
1313/1313 [==============================] - 258s 194ms/step - loss: 0.8930 - accuracy: 0.5742
Epoch 2/4
1313/1313 [==============================] - 253s 192ms/step - loss: 0.8413 - accuracy: 0.6037
Epoch 3/4
1313/1313 [==============================] - 254s 193ms/step - loss: 0.8035 - accuracy: 0.6311
Epoch 4/4
1313/1313 [==============================] - 252s 192ms/step - loss: 0.7792 - accuracy: 0.6499


NameError: ignored

In [20]:
#GRU_model = get_trained_GRU_model(data_x,data_y,embedding_matrix,word_count,max_sentence_length)
predictions_GRU_model = get_predictions_single_model(GRU_model,dev_x)
GRU_avg_recall = calculate_avg_recall(predictions_GRU_model,dev_y)

print(GRU_avg_recall)

NameError: ignored

In [ ]:
RNN_model1 = get_RNN_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length)
predictions_RNN_model1 = get_predictions_single_model(RNN_model1,dev_x)
RNN1_avg_recall = calculate_avg_recall(predictions_RNN_model1,dev_y)

print(RNN1_avg_recall)

Epoch 1/2
1178/1178 [==============================] - 75s 61ms/step - loss: 1.2649 - accuracy: 0.4081
Epoch 2/2
1178/1178 [==============================] - 71s 60ms/step - loss: 1.0373 - accuracy: 0.4455
Epoch 1/4
1178/1178 [==============================] - 74s 62ms/step - loss: 1.0269 - accuracy: 0.4485
Epoch 2/4
1178/1178 [==============================] - 73s 62ms/step - loss: 1.0046 - accuracy: 0.4710
Epoch 3/4
1178/1178 [==============================] - 73s 62ms/step - loss: 0.9929 - accuracy: 0.4820
Epoch 4/4
1178/1178 [==============================] - 73s 62ms/step - loss: 0.9851 - accuracy: 0.4926


NameError: ignored

In [ ]:
predictions_RNN_model1 = get_predictions_single_model(RNN_model1,dev_x)
RNN1_avg_recall = calculate_avg_recall(predictions_RNN_model1,dev_y)

[[0.08120309 0.4658849  0.45291203]
 [0.06992232 0.51026994 0.4198077 ]
 [0.16379468 0.37908384 0.4571215 ]
 ...
 [0.06840565 0.44109267 0.49050176]
 [0.07189697 0.58978003 0.33832297]
 [0.06610809 0.5462828  0.38760912]]
[1, 1, 2, 2, 1, 1, 0, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 0, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 0, 2, 0, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 

In [ ]:
LSTM_avg_model = get_modified_LSTM_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length,dev_x,dev_y)
predictions_LSTM_model2 = get_predictions_single_model(LSTM_avg_model,dev_x)
LSTM2_avg_recall = calculate_avg_recall(predictions_LSTM_model2,dev_y)

print(LSTM2_avg_recall)

Epoch 1/4
998/998 [==============================] - 220s 217ms/step - loss: 0.9827 - accuracy: 0.5060 - val_loss: 0.8160 - val_accuracy: 0.6228
Epoch 2/4
998/998 [==============================] - 215s 216ms/step - loss: 0.8529 - accuracy: 0.5943 - val_loss: 0.7839 - val_accuracy: 0.6391
Epoch 3/4
998/998 [==============================] - 215s 216ms/step - loss: 0.8071 - accuracy: 0.6231 - val_loss: 0.7954 - val_accuracy: 0.6359
Epoch 4/4
998/998 [==============================] - 213s 214ms/step - loss: 0.7737 - accuracy: 0.6453 - val_loss: 0.7809 - val_accuracy: 0.6558
[[0.02550137 0.32859847 0.6459002 ]
 [0.00947395 0.7543254  0.23620068]
 [0.07120452 0.68945014 0.23934539]
 ...
 [0.60750157 0.3498351  0.04266333]
 [0.00767831 0.22750555 0.7648161 ]
 [0.00581334 0.08286975 0.9113169 ]]
[2, 1, 1, 2, 1, 2, 2, 1, 1, 0, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 0, 1,

In [ ]:
LSTM_model2 = get_LSTM_Attn_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length,dev_x,dev_y)
predictions_LSTM_model2 = get_predictions_single_model(LSTM_model2,dev_x)
LSTM2_avg_recall = calculate_avg_recall(predictions_LSTM_model2,dev_y)

print(LSTM2_avg_recall)

Epoch 1/4
998/998 [==============================] - 215s 212ms/step - loss: 0.9792 - accuracy: 0.5069 - val_loss: 0.8228 - val_accuracy: 0.6137
Epoch 2/4
998/998 [==============================] - 210s 211ms/step - loss: 0.8470 - accuracy: 0.5984 - val_loss: 0.8133 - val_accuracy: 0.6281
Epoch 3/4
998/998 [==============================] - 211s 211ms/step - loss: 0.7959 - accuracy: 0.6303 - val_loss: 0.7668 - val_accuracy: 0.6548
Epoch 4/4
998/998 [==============================] - 211s 211ms/step - loss: 0.7702 - accuracy: 0.6503 - val_loss: 0.7654 - val_accuracy: 0.6586
[[0.02980436 0.35544866 0.614747  ]
 [0.01395734 0.7022152  0.28382748]
 [0.05703976 0.7900331  0.15292713]
 ...
 [0.4739956  0.45580205 0.07020238]
 [0.01513075 0.27863613 0.70623314]
 [0.00319186 0.07080527 0.92600286]]
[2, 1, 1, 2, 1, 2, 2, 1, 1, 0, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 0, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 0, 1,

In [ ]:
CNN_model = get_trained_CNN_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length)
predictions_CNN_model = get_predictions_single_model(CNN_model,dev_x)
CNN_avg_recall = calculate_avg_recall(predictions_CNN_model,dev_y)

print(CNN_avg_recall)

getting weights
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 31, 50)       1557650     input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 1, 31, 50, 1) 0           embedding_3[0][0]                
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 1, 30, 1, 50) 5050        reshape[0][0]                    
_______________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/2
1178/1178 [==============================] - 18s 15ms/step - loss: 1.0761 - accuracy: 0.3766
Epoch 2/2
1178/1178 [==============================] - 18s 15ms/step - loss: 0.8246 - accuracy: 0.6542
Epoch 1/4
1178/1178 [==============================] - 18s 15ms/step - loss: 0.6724 - accuracy: 0.7307
Epoch 2/4
1178/1178 [==============================] - 18s 15ms/step - loss: 0.5552 - accuracy: 0.7935
Epoch 3/4
1178/1178 [==============================] - 18s 15ms/step - loss: 0.4240 - accuracy: 0.8584
Epoch 4/4
1178/1178 [==============================] - 18s 15ms/step - loss: 0.3106 - accuracy: 0.9091
[[0.00700608 0.03088577 0.9621082 ]
 [0.00755632 0.09028471 0.902159  ]
 [0.00238726 0.40533137 0.59228134]
 ...
 [0.00127735 0.95611185 0.04261083]
 [0.08983183 0.9087623  0.0014059 ]
 [0.00301399 0.99525625 0.00172973]]
[2, 2, 2, 1, 1, 1, 1, 0, 2, 1, 2, 2, 2, 1, 0, 2, 1, 1, 1, 2, 0, 2, 2, 1, 2, 2, 0, 1, 2, 0, 0, 1, 0, 2, 1, 2, 0, 1, 2, 2, 2, 1, 0, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 

In [23]:
test_file = open("/content/drive/MyDrive/Datasets/test_2016",'r',encoding='utf-8')
text = test_file.read()
test_file.close()

text = replace_emojis(text)
word_sentences = split_into_words(text)
sentences,sentence_classes,sentence_ids = seperate_classes_ids_sentences(word_sentences)
sentences = remove_punctuations(sentences)
sentences = remove_recurring_characters(sentences)
sentence_vectors = get_test_sentence_vectors(sentences,word_count)
test_x = pad_input_vectors(sentence_vectors,max_sentence_length)
test_y = get_test_class_labels(sentence_classes)

predictions_LSTM_model1 = get_predictions_single_model(GRU_model,test_x)
LSTM1_avg_recall = calculate_avg_recall(predictions_LSTM_model1,test_y)

# predictions_LSTM_model2 = get_predictions_single_model(LSTM_avg_model,test_x)
# LSTM2_avg_recall = calculate_avg_recall(predictions_LSTM_model2,test_y)

# predictions_LSTM_Attn_model = get_predictions_single_model(LSTM_model2,test_x)
# LSTM2_avg_recall = calculate_avg_recall(predictions_LSTM_Attn_model ,test_y)

# predictions_CNN_model = get_predictions_single_model(CNN_model,test_x)
# CNN_avg_recall = calculate_avg_recall(predictions_CNN_model,test_y)

#print(LSTM1_avg_recall,LSTM2_avg_recall,CNN_avg_recall)
print(LSTM1_avg_recall)

[619950566786113536, 619969366986235905, 619971047195045888, 619974445185302528, 619987808317407232, 619994586182619136, 619998879447162881, 619999109991284737, 620013074272137216, 620015047889154048, 620015649151021056, 620017122081685504, 620023409901043712, 620024331322699776, 620027768131203072, 620031929182621696, 620033827046596608, 620035146083880961, 620040573395701760, 620045069605691392, 620045849373093892, 620045999977963520, 620056364149309441, 620056577341562884, 620060010396041216, 620063502682599425, 620064098731073536, 620067692066828288, 620068767930802177, 620073648255168512, 620076360040583169, 620080951708651520, 620084129640546304, 620099074679701509, 620111581230149632, 620125231164715008, 620131126888132608, 620133093005856768, 620133289668407296, 620149443191533568, 620152459676282880, 620156193026252801, 620163205327773696, 620166981614968832, 620170215507292160, 620173983179231232, 620192443204546560, 620201635013337088, 620208590951960576, 620237663170228224,

In [ ]:
import pandas as pd
output_df = pd.DataFrame({'predictions':predictions_LSTM_model1},index=None)
output_df.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Project/Results/LSTM_std_2016.csv',index=False) 

In [ ]:
ensemble_predictions = []
for i in range(len(predictions_LSTM_model1)):
  votes = [0]*4
  votes[predictions_LSTM_model1[i]] += 1
  votes[predictions_LSTM_model2[i]] += 1
  votes[predictions_LSTM_Attn_model[i]] += 1
  votes[predictions_CNN_model[i]] += 1
  for j in range(3):
    if votes[j] == max(votes):
      ensemble_predictions.append(j)
      break
ensemble_avg_recall = calculate_avg_recall(ensemble_predictions,test_y)

NameError: ignored

In [ ]:
print(LSTM_model1.evaluate(test_x,test_y))
print(LSTM_model2.evaluate(test_x,test_y))
print(CNN_model.evaluate(test_x,test_y))

In [ ]:
print(test_y.shape)
print(CNN_model.evaluate(test_x,test_y))

(20632,)
645/645 [==============================] - 22s 33ms/step - loss: 0.8617 - accuracy: 0.5303
[0.8616921305656433, 0.5303412079811096]


In [ ]:
## EXTRACT POSITIVE AND NEGATIVE TWEETS BASED ON EMOTICON
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ADL/ADL_1/distnt_training_1.6.csv",encoding='ISO-8859-1',index_col=['No.','Id','Date','misc1','misc2','tweet'])
print(df.columns)
tweets = df["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"]
print(tweets[0:10])
labels = df['0']
print(labels[0:10])

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')
0    is upset that he can't update his Facebook by ...
1    @Kenichan I dived many times for the ball. Man...
2      my whole body feels itchy and like its on fire 
3    @nationwideclass no, it's not behaving at all....
4                        @Kwesidei not the whole crew 
5                                          Need a hug 
6    @LOLTrish hey  long time no see! Yes.. Rains a...
7                 @Tatiana_K nope they didn't have it 
8                            @twittera que me muera ? 
9          spring break in plain city... it's snowing 
Name: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D, dtype: object
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    

In [ ]:
for i in range(len(tweets)):
  tweets[i] = replace_emojis(tweets[i])
  tweets[i] = split_into_words(tweets[i])
print(tweets[0:10])